<a href="https://colab.research.google.com/github/JeMinYoo/Aiffel-Quest/blob/master/Main_quest/Quest05/DLThon_ToiToy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STEP 1 데이터 전처리 및 토크나이징


## 1.1 한국어 단발성 대화 데이터셋

### 1.1.1 KoBERT

In [ ]:
import pandas as pd
# [AI Hub] 감정 분류를 위한 대화 음성 데이터셋
chatbot_concat_data  = pd.read_excel('/content/gdrive/MyDrive/Colab Notebooks/concat_talks.xlsx')

chatbot_concat_data.sample(n=10)
# 7개의 감정 class → 숫자
chatbot_data.loc[(chatbot_data['Emotion'] == "공포"), 'Emotion'] = 0  #공포 => 0
chatbot_data.loc[(chatbot_data['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 => 1
chatbot_data.loc[(chatbot_data['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
chatbot_data.loc[(chatbot_data['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
chatbot_data.loc[(chatbot_data['Emotion'] == "중립"), 'Emotion'] = 4  #중립 => 4
chatbot_data.loc[(chatbot_data['Emotion'] == "행복"), 'Emotion'] = 5  #행복 => 5
chatbot_data.loc[(chatbot_data['Emotion'] == "혐오"), 'Emotion'] = 6  #혐오 => 6

data_list = []
for q, label in zip(chatbot_concat_data['Sentence'], chatbot_concat_data['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data)
print(data_list[:10])

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size = 0.2, shuffle = True, random_state = 32)

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# BERTDataset : 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
tok = tokenizer.tokenize

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

# torch 형식의 dataset을 만들어 입력 데이터셋의 전처리 마무리
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)

## 1.2 감성 대화 말뭉치

# STEP 2 베이스 모델

## 2.1 KoBERT

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
device = torch.device("cuda:0")
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
max_len = 64
batch_size = 128
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  3e-5

In [ ]:
# BERTDataset : 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
tok = tokenizer.tokenize

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

# torch 형식의 dataset을 만들어 입력 데이터셋의 전처리 마무리
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 7,   # 감정 클래스 수로 조정
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# BERT  모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate = 0.7).to(device)

# optimizer와 schedule 설정
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.01}
]


optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)

# calc_accuracy : 정확도 측정을 위한 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

결과
train acc는 0.85
test acc가 0.55

## 2.2 KcELECTRA

## 2.3 KoNLPy

# STEP 3 모델 성능 개선

## 3.1 KoBERT

배치 사이즈와 학습률, 드롭아웃을 조정해 봤으나 개선되지 않았다.
데이터셋의 문제로 판단하여 기존 데이터셋(한국어 단발성 대화)와 감성 대화 말뭉치를 합치고 7개의 감정을 5개의 감정으로 변경하였고 추가로 데이터 증강 진행

In [ ]:
import pandas as pd
# [AI Hub] 감정 분류를 위한 대화 음성 데이터셋
chatbot_concat_data  = pd.read_excel('/content/gdrive/MyDrive/Colab Notebooks/concat_talks.xlsx')

chatbot_concat_data.sample(n=10)
# 5개의 감정 class → 숫자
chatbot_concat_data.loc[(chatbot_concat_data['Emotion'] == "불안"), 'Emotion'] = 0  #불안 => 0
chatbot_concat_data.loc[(chatbot_concat_data['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 => 1
chatbot_concat_data.loc[(chatbot_concat_data['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
chatbot_concat_data.loc[(chatbot_concat_data['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
chatbot_concat_data.loc[(chatbot_concat_data['Emotion'] == "행복"), 'Emotion'] = 4  #행복 => 4

data_list = []
for q, label in zip(chatbot_concat_data['Sentence'], chatbot_concat_data['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data)
print(data_list[:10])

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
from textattack.augmentation import WordNetAugmenter

augmenter = WordNetAugmenter()
augmented_data_list = []

# 원본 데이터 증강
for data in data_list:
    sentence, label = data
    augmented_sentences = augmenter.augment(sentence)
    for augmented_sentence in augmented_sentences:
        augmented_data_list.append([augmented_sentence, label])

# 증강된 데이터를 원본 데이터에 추가
data_list += augmented_data_list

# 다시 train & test 데이터로 나누기
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=32)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 5,   # 감정 클래스 수로 조정
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# BERT  모델 불러오기
model2 = BERTClassifier(bertmodel,  dr_rate = 0.7).to(device)

# optimizer와 schedule 설정
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model2.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model2.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.01}
]


optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)

# calc_accuracy : 정확도 측정을 위한 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model2.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model2(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model2.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model2.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model2(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

결과
train acc는 0.82
test acc가 0.74로 상승

In [ ]:
import pandas as pd
# [AI Hub] 감정 분류를 위한 대화 음성 데이터셋
chatbot_concat_data  = pd.read_excel('/content/gdrive/MyDrive/Colab Notebooks/concat_talks.xlsx')

chatbot_concat_data.sample(n=10)
# 5개의 감정 class → 숫자
chatbot_concat_data.loc[(chatbot_concat_data['Emotion'] == "불안"), 'Emotion'] = 0  #불안 => 0
chatbot_concat_data.loc[(chatbot_concat_data['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 => 1
chatbot_concat_data.loc[(chatbot_concat_data['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
chatbot_concat_data.loc[(chatbot_concat_data['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
chatbot_concat_data.loc[(chatbot_concat_data['Emotion'] == "행복"), 'Emotion'] = 4  #행복 => 4

data_list = []
for q, label in zip(chatbot_concat_data['Sentence'], chatbot_concat_data['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data)
print(data_list[:10])

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
from textattack.augmentation import WordNetAugmenter

augmenter = WordNetAugmenter()
augmented_data_list = []

# 원본 데이터 증강
for data in data_list:
    sentence, label = data
    augmented_sentences = augmenter.augment(sentence)
    for augmented_sentence in augmented_sentences:
        augmented_data_list.append([augmented_sentence, label])

# 증강된 데이터를 원본 데이터에 추가
data_list += augmented_data_list

# 다시 train & test 데이터로 나누기
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=32)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 5,   # 감정 클래스 수로 조정
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# BERT  모델 불러오기
model2 = BERTClassifier(bertmodel,  dr_rate = 0.7).to(device)

# optimizer와 schedule 설정
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model2.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model2.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.01}
]


optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)

# calc_accuracy : 정확도 측정을 위한 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model2.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model2(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model2.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model2.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model2(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

결과

train acc는 0.86
test acc가 0.80로 상승

## 3.2 KcELECTRA

## 3.3 KoNLPy

# STEP 4 최종 모델

## 4.1 KcELECTRA

# STEP 5 FastAPI

In [ ]:
import uvicorn
from fastapi import FastAPI, Request, HTTPException, File, UploadFile, Query
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from starlette.middleware.base import BaseHTTPMiddleware
from PIL import Image
import io
import logging
import requests
from openai import OpenAI   # GPT-4o mini 사용 위해 추가
from pathlib import Path    # TTS-1 오디오 파일 생성 경로 설정을 위해 추가
from pydantic import BaseModel  # Request 데이터의 유효성 검사 및 문서 자동화를 위해 추가
from typing import Optional     # Python의 타입 힌팅을 위해 추가. (Optional을 사용하면 값이 있을 수도 있고 없을 수도 있음을 의미. 즉, 필수가 아닌 선택을 의미함)
from transformers import AutoTokenizer, TFAutoModel, TFAutoModelForSequenceClassification   # KcELECTRA 모델 사용 위해 추가
import tensorflow as tf
import xmltodict    # XML 파싱을 위해 추가(maniadb.com API)
from urllib.parse import quote, urlencode, urlunparse   # URL 인코딩(특히, 한글)을 위해 추가
import random


# FastAPI 앱 생성
app = FastAPI()

# CORS configuration
origins = ["*"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 저장해 둔 KcELECTRA 모델과 토크나이저 로드
num_labels = 5  # 분석할 감정 레이블 갯수
try:
    tokenizer = AutoTokenizer.from_pretrained("./kcelectra_fine_tuned/tokenizer/")
    electra_model = TFAutoModelForSequenceClassification.from_pretrained(
        './kcelectra_fine_tuned/',
        num_labels=num_labels)
except Exception as e:
    logger.error("Failed to load model: %s", e)
    raise

# A simple example of a GET request
@app.get("/")
async def read_root():
    logger.info("Root URL was requested")
    return "안녕하세요. 저희는 아이펠 DLthon에 참여중인 팀 ToiToy예요. 만나서 반갑습니다."


# POST 요청 데이터의 유효성 검사와 문서 자동화를 위해 Pydantic 모델 상속받아 커스텀 클래스 정의
# 즉, POST 요청을 통해 서버로 전송될 데이터의 형식이라고 보면 됨.
class TextRequest(BaseModel):
    text: str

# [테스트] 문자열을 전달받아 간단한 처리 후 JSON 형식으로 응답
@app.post("/dlthon_test/")
async def process_text(request: TextRequest):
    text = request.text

    # 문자열 처리 로직 (예: 문자열을 대문자로 변환)
    processed_text = text.upper()

    # JSON 응답 생성
    response = {
        "original_text": text,
        "processed_text": processed_text
    }

    # [참고] FastAPI는 응답 대상이 pydantic 객체, dict, list 타입인 경우에는 JSON 직렬화를 자동으로 함. (예: 아래 return response 처럼)
    # (물론, Content-Type 헤더도 'application/json'으로, status code도 '200 OK'로 자동 설정됨)
    # 반면, FastAPI의 공식 응답 객체인 JSONResponse 클래스를 사용하면 응답 본문, 상태 코드, 헤더 등을 명시적으로 지정할 수 있음.
    # return response
    return JSONResponse(content=response)


# 문자열을 전달받아 AI 모델에 의해 감정 분석 후 분석 결과를 리턴
@app.post("/sentiment/")
async def analyze_sentiment(request: TextRequest):
    text = request.text     # POST 요청으로 전달된 문자열

    # 감정 분석
    inputs = tokenizer(text, return_tensors="tf")
    logits = electra_model(**inputs).logits
    predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])    # [0]은 배치 차원이 있는 경우 첫번째 샘플을 의미함.
    predicted_label = electra_model.config.id2label[predicted_class_id]
    # Softmax 변환
    probabilities = tf.nn.softmax(logits, axis=-1)
    probabilities_list = probabilities.numpy()[0].tolist()
    predicted_output = probabilities_list

    # # 테스트용
    # predicted_output = ""
    # predicted_label = "슬픔"

    # ChatGPT-4o-mini에게 요청
    try:
        MODEL="gpt-4o-mini"     # GPT 모델 지정
        client = OpenAI()

        # 사용자가 POST 요청으로 전달한 문장 자체를 기준으로 프롬프트 작성
        if predicted_label == '행복':
            system_content = f"너는 심리적 어려움을 가지고 있는 사람들을 돕는 전문 상담가야. 내가 입력하는 문장에 대한 공감과 응원의 말을 작성해 줘. 전문 상담가의 느낌이 나도록 가볍지 않은 어투로 작성해 주고 늘 존대말로 답변해 줘. 맨 처음 '안녕하세요', '안녕' 같은 인사말은 하지 않아도 되."
        else:
            system_content = f"너는 심리적 어려움을 가지고 있는 사람들을 돕는 전문 상담가야. 내가 입력하는 문장에 대한 공감과 위로, 응원의 말을 작성해 줘. 전문 상담가의 느낌이 나도록 가볍지 않은 어투로 작성해 주고 늘 존대말로 답변해 줘. 맨 처음 '안녕하세요', '안녕' 같은 인사말은 하지 않아도 되."

        # AI 모델이 분석한 결과 레이블 값을 기준으로 프롬프트 작성
        # --> [결정적 단점] AI 모델이 분석한 결과 레이블 값이 이상하면 ChatGPT의 답변 역시 이상한 결과가 나옴.
        # if predicted_label == '행복':
        #     content = f"{predicted_label}의 감정을 가지고 있는 사람에게 도움이 되도록 공감과 응원의 말을 작성해 줘. 전문 상담가의 느낌이 나도록 가볍지 않은 어투로 작성해 주고 늘 존대말로 답변해 줘. 맨 처음 '안녕하세요', '안녕' 같은 인사말은 하지 않아도 되."
        # else:
        #     content = f"{predicted_label}의 감정을 가지고 있는 사람에게 도움이 되도록 공감과 위로, 응원의 말을 작성해 줘. 전문 상담가의 느낌이 나도록 가볍지 않은 어투로 작성해 주고 늘 존대말로 답변해 줘. 맨 처음 '안녕하세요', '안녕' 같은 인사말은 하지 않아도 되."

        completion = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_content},
                # {"role": "user", "content": content}      # AI 모델이 분석한 결과 레이블 기준으로 요청하는 경우
                {"role": "user", "content": text}       # 사용자가 POST 요청으로 전달한 문장을 그대로 사용해서 요청하는 경우
            ]
        )

        answer = completion.choices[0].message.content

        if predicted_label == '행복':
            answer += "\n\n앞으로도 더욱 멋진 날들이 이어지기를 바라는 마음으로 노래를 몇 곡 준비해 봤습니다. 멋진 노래와 함께 하루를 잘 마무리하길 바랄게요!"
        else:
            answer += "\n\n조금이나마 도움이 되길 바라는 마음으로 노래를 몇 곡 준비해 봤습니다. 때로는 몇 마디 말보다 노래 하나가 더 힘이 될 때도 많으니까요."
    except Exception as e:
        logger.error("Prediction failed: %s", e)
        raise HTTPException(status_code=500, detail="Internal Server Error")

    # JSON 응답 생성
    response = {
        "probabilities": predicted_output,
        "predicted_label": predicted_label,
        "answer": answer
    }

    return JSONResponse(content=response)


# 감정에 해당하는 단어를 전달받아 공감과 위로, 응원이 되는 문장을 생성해서 리턴
@app.get('/comfort/')
async def generate_words_of_comfort(sentiment: str = Query(...)):
    # ChatGPT-4o-mini에게 요청
    try:
        MODEL="gpt-4o-mini"     # GPT 모델 지정
        client = OpenAI()

        if sentiment == '행복':
            content = f"{sentiment}의 감정을 가지고 있는 사람에게 도움이 되도록 공감과 응원의 말을 작성해 줘. 친근감이 느껴지도록 친구와 대화하는 느낌의 반말로 작성해 줘. 맨 처음 '안녕하세요', '안녕' 같은 인사말은 하지 않아도 되."
        else:
            content = f"{sentiment}의 감정을 가지고 있는 사람에게 도움이 되도록 공감과 위로, 응원의 말을 작성해 줘. 친근감이 느껴지도록 친구와 대화하는 느낌의 반말로 작성해 줘. 맨 처음 '안녕하세요', '안녕' 같은 인사말은 하지 않아도 되."

        completion = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": "너는 심리적 어려움을 가지고 있는 사람들을 돕는 전문 상담가야."},
                {"role": "user", "content": content}
            ]
        )

        answer = completion.choices[0].message.content
    except Exception as e:
        logger.error("Prediction failed: %s", e)
        raise HTTPException(status_code=500, detail="Internal Server Error")

    # JSON 응답 생성
    response = {
        "answer": answer
    }

    return JSONResponse(content=response)


# 감정에 해당하는 단어를 전달받아 그에 해당하는 음악 리스트를 리턴 (maniadb.com API 활용)
# mainadb.com API에서는 응답을 XML 형식으로 전달하기 때문에 XML을 파싱해서 필요한 엘리먼트만 추출해 JSON 형식으로 변환해야 함!
@app.get('/music/')
async def generate_music_list(sentiment: str = Query(...)):
    # 한글 단어와 쿼리 파라미터
    search_term = sentiment  # 감정 단어('공포', '분노', '불안', '슬픔', '행복')
    base_url = "https://www.maniadb.com/api/search/"
    query_params = {
        "sr": "song",
        "display": 5,
        "key": "example",
        "v": "0.5"
    }

    # 한글 단어를 URL 인코딩
    encoded_search_term = quote(search_term)
    # 쿼리 파라미터를 URL 인코딩
    encoded_query_params = urlencode(query_params)

    # 전체 URL 구성(인코딩 처리된 최종 URL)
    url = f"{base_url}{encoded_search_term}/?{encoded_query_params}"

    try:
        # XML 데이터 요청
        response = requests.get(url)
        response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

        # XML 데이터를 JSON으로 변환
        xml_data = response.text
        json_data = xmltodict.parse(xml_data)

        # 'item' 태그만 추출
        items = json_data['rss']['channel']['item']

        # item에서 필요한 데이터만 추출
        results = []

        for item in items:
            result = {
                'title': item.get('title', ''),
                'artist': item.get('maniadb:artist', {}).get('name', ''),  # name 태그는 maniadb:artist 안에 있음
                'link': item.get('link', ''),
                'album_title': item.get('maniadb:album', {}).get('title', ''),  # title 태그는 maniadb:album 안에 있음
                'album_link': item.get('maniadb:album', {}).get('link', ''),  # link 태그는 maniadb:album 안에 있음
                'album_image': item.get('maniadb:album', {}).get('image', '')  # image 태그는 maniadb:album 안에 있음
            }
            results.append(result)

        # JSON 응답 반환
        return JSONResponse(content={"items": results})
    # except requests.exceptions.RequestException as e:
    #     return JSONResponse(content={"error": str(e)}, status_code=500)
    except Exception as e:
        logger.error("maniadb.com API call failed: %s", e)
        print(f"An error occurred: {str(e)}")
        raise HTTPException(status_code=500, detail=f"Internal Server Error")


# 감정에 해당하는 단어를 전달받아 그에 해당하는 음악 리스트를 리턴 (maniadb.com API 활용)
# mainadb.com API에서는 응답을 XML 형식으로 전달하기 때문에 XML을 파싱해서 필요한 엘리먼트만 추출해 JSON 형식으로 변환해야 함!
# [중요] /music/과의 차이점 : /music2/는 감정에 해당하는 단어와 연관된 단어 중에서 랜덤하게 선택해 API 호출하는 방식.
# API 호출이 5번 이루어지기 때문에 응답 시간이 좀 더 오래 걸림! (평균 10초 내외. /music/은 3초)
@app.get('/music2/')
async def generate_music_list2(sentiment: str = Query(...)):
    # 한글 단어와 쿼리 파라미터
    search_term = sentiment  # 감정 단어('공포', '분노', '불안', '슬픔', '행복')
    base_url = "https://www.maniadb.com/api/search/"
    query_params = {
        "sr": "song",
        "display": 1,   # API 호출당 검색 노래 수(호출할 때마다 키워드로 사용되는 단어가 달라져서 일부러 1로 설정함)
        "key": "example",
        "v": "0.5"
    }

    # 감정별 연관 단어 리스트를 포함하는 딕셔너리
    emotion_words = {
        '공포': [
            '용기', '안전', '평화', '극복', '보호', '신뢰', '빛', '온기', '포옹', '조용', '고요', '평온', '명상', '호흡', '기도'
        ],
        '분노': [
            '용서', '평정', '이해', '생각', '차분', '조절', '수용', '인내', '연민', '관용', '화해', '평화', '해방', '안도', '명상', '호흡', '자연', '물', '바람', '흙', '공기'
        ],
        '불안': [
            '안정', '평온', '편안', '차분', '긍정', '희망', '용기', '믿음', '조절', '호흡', '명상', '자연', '온기', '포옹', '안전', '보호', '휴식'
        ],
        '슬픔': [
            '위로', '공감', '위안', '치유', '회복', '희망', '긍정', '온기', '포옹', '사랑', '기억', '추억', '시간', '변화', '용기', '미소', '행복'
        ],
        '행복': [
            '행복', '기쁨', '감사', '만족', '사랑', '평화', '행운', '즐거움', '환희', '축복', '긍정', '희망', '열정', '활력', '성장', '발전', '성공', '자유', '만족'
        ]
    }

    # GET 요청시 넘어온 search_term 감정과 연관된 단어 중에서 랜덤하게 5개만 뽑기
    search_random_words = random.sample(emotion_words[search_term], 5)
    print(search_random_words)

    # 한글 단어 URL 인코딩 : 랜덤하게 추출된 5개 단어 각각에 대해 처리해서 리스트로 저장
    encoded_search_term_list = [quote(word) for word in search_random_words]

    # 쿼리 파라미터를 URL 인코딩 : 공통이라 한 번만 하면 됨.
    encoded_query_params = urlencode(query_params)

    # 인코딩된 한글 단어 리스트를 순회하면서 최종 url 리스트 생성
    url_list = []
    for encoded_search_term in encoded_search_term_list:
        url_list.append(f"{base_url}{encoded_search_term}/?{encoded_query_params}")

    try:
        # 5개의 음악 리스트를 저장
        results = []

        # 5개의 url 리스트 각각을 순회하면서 처리 --> 총 5회의 API 호출 발생함.
        for url in url_list:
            # XML 데이터 요청
            response = requests.get(url)
            response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

            # XML 데이터를 JSON으로 변환
            xml_data = response.text
            json_data = xmltodict.parse(xml_data)

            # 'item' 태그만 추출
            # [중요] 음악 하나만 가져오는 상황이라 [0] 없이 그냥 ['item']으로 끝나도 됨. (print문으로 직접 확인함)
            item = json_data['rss']['channel']['item']

            result = {
                'title': item.get('title', ''),
                'artist': item.get('maniadb:artist', {}).get('name', ''),  # name 태그는 maniadb:artist 안에 있음
                'link': item.get('link', ''),
                'album_title': item.get('maniadb:album', {}).get('title', ''),  # title 태그는 maniadb:album 안에 있음
                'album_link': item.get('maniadb:album', {}).get('link', ''),  # link 태그는 maniadb:album 안에 있음
                'album_image': item.get('maniadb:album', {}).get('image', '')  # image 태그는 maniadb:album 안에 있음
            }
            results.append(result)

        # JSON 응답 반환
        return JSONResponse(content={"items": results})
    # except requests.exceptions.RequestException as e:
    #     return JSONResponse(content={"error": str(e)}, status_code=500)
    except Exception as e:
        logger.error("maniadb.com API call failed: %s", e)
        print(f"An error occurred: {str(e)}")
        raise HTTPException(status_code=500, detail=f"Internal Server Error")


# 앱 실행
if __name__ == "__main__":
    uvicorn.run("server_fastapi_dlthon:app",
            reload= True,   # Reload the server when code changes
            host="127.0.0.1",   # Listen on localhost
            port=5000,   # Listen on port 5000
            log_level="info"   # Log level
            )